In [43]:
import pandas as pd
import os
import seaborn as sns
import shutil

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

# ? Se usa para generar gráficos
import matplotlib.pyplot as plt

# ? Se usa para determina la fecha de ejecución del proceso dentro del export de los resultados estadísticos
from datetime import datetime

# ? Conexión a BD Postgres. Se usa dado que pandas no acepta sino este paquete para la conexión.
from sqlalchemy import create_engine, text

from datetime  import datetime as dt

from arcgis.features import GeoAccessor, GeoSeriesAccessor
import arcpy

from pathlib import Path

# ** Librerías propias

In [44]:
RUTA_REL_TERRENOS_UI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\relacion_terreno_ui"
RUTA_UI = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\O_Apoyo_Construccion_UIntervencion\UI_Finales\UI_Finales.gdb\UI_Unificadas\UI_Unificadas"
RUTA_CAPA_TERRENOS = r"C:\docsProyectos\5.RAISS\2024.0.RAISS_Lote_4\6.Hitos\E1_Alistamiento_Diagnostico\3_Disposicion\1.BD_Consolidada\BD_Consolidada_Lote4.gdb\Analitica_UT_Consolidada\TERRENO_UNIFICADO_w_GIS"

estandarizacion_columnas_terrenos = ['codigo', 'zona_predio', 'area_ha_cmt12','SHAPE']
estandarizacion_columnas_terreno_hito = ['codigo','ID_UI','Zona_UI','Departamento_UI','Municipio_UI','Meta_Hito']


In [45]:
df_rel_terrenos_ui = pd.DataFrame.spatial.from_table(RUTA_REL_TERRENOS_UI)
df_ui = pd.DataFrame.spatial.from_featureclass(RUTA_UI)

df_terrenos = pd.DataFrame.spatial.from_featureclass(RUTA_CAPA_TERRENOS)

# TODO: Generacion Campo Zona
df_terrenos = df_terrenos.copy()

df_terrenos['codigo_zona'] = df_terrenos['codigo'].str[5:7]
df_terrenos['zona_predio'] = None

df_terrenos.loc[df_terrenos['codigo_zona'] == '01', 'zona_predio'] = 'Urbano'
df_terrenos.loc[df_terrenos['codigo_zona'] == '00', 'zona_predio'] = 'Rural'
df_terrenos.loc[(df_terrenos['codigo_zona'] != '00') & (df_terrenos['codigo_zona'] != '01'), 'zona_predio'] = 'Corregimientos Urbanos'

df_terrenos = df_terrenos[estandarizacion_columnas_terrenos]

In [46]:
pd_terreno_hito = pd.merge(left=df_rel_terrenos_ui,
    right=df_ui,
    left_on='id_ui',
    right_on='ID_UI',
    how='left')

df_terreno_hito = pd_terreno_hito[estandarizacion_columnas_terreno_hito]

In [47]:
df_estadistico_terreno_hito =pd.merge(left=df_terreno_hito,
    right=df_terrenos,
    on='codigo',
    how='inner')

In [48]:
df_estadistico_terreno_hito

,codigo,ID_UI,Zona_UI,Departamento_UI,Municipio_UI,Meta_Hito,zona_predio,area_ha_cmt12,SHAPE
0,134420000000000030566000000000,ZI_MB_08,Rural,BOLÍVAR,MARÍA LA BAJA,Hito 3,Rural,0.052848,"{""rings"": [[[4738159.3177, 2657370.8461000007]..."
1,134420000000000030144000000000,<NA>,<NA>,<NA>,<NA>,<NA>,Rural,5.069531,"{""rings"": [[[4736779.6503, 2662389.763699999],..."
2,134420000000000030134000000000,<NA>,<NA>,<NA>,<NA>,<NA>,Rural,12.018001,"{""rings"": [[[4736619.7828, 2662306.5286], [473..."
3,134420000000000030166000000000,ZI_MB_08,Rural,BOLÍVAR,MARÍA LA BAJA,Hito 3,Rural,0.355461,"{""rings"": [[[4738235.3088, 2657335.172700001],..."
4,134420000000000030192000000000,ZI_MB_08,Rural,BOLÍVAR,MARÍA LA BAJA,Hito 3,Rural,0.507928,"{""rings"": [[[4738387.1115, 2656714.7664], [473..."
...,...,...,...,...,...,...,...,...,...
39851,080780100000001070007000000000,ZI_BA_06_0,Urbana,ATLÁNTICO,BARANOA,Hito 5,Urbano,0.010442,"{""rings"": [[[4789783.365, 2751754.2195999995],..."
39852,080780100000001070021000000000,ZI_BA_06_0,Urbana,ATLÁNTICO,BARANOA,Hito 5,Urbano,0.014832,"{""rings"": [[[4789789.4954, 2751753.0899], [478..."
39853,080780100000000090013000000000,ZI_BA_06_0,Urbana,ATLÁNTICO,BARANOA,Hito 5,Urbano,0.004797,"{""rings"": [[[4790916.6702, 2752244.5452999994]..."
39854,080780100000000090006000000000,ZI_BA_06_0,Urbana,ATLÁNTICO,BARANOA,Hito 5,Urbano,0.015991,"{""rings"": [[[4790924.6077, 2752240.383300001],..."
